#### create_hdfs.py in NAF (revisited)  

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from matplotlib import ticker, cm
from matplotlib.colors import LogNorm
import h5py
import uproot
import logging
import os


#Open branch from the ntuple (look at the number after ;)
ntuple = uproot.open('calo_hits-test.root')["photonSIM;1"]


Nevents = 1000

## Select calorimeter hits and MC particle from the braches
x = ntuple.array("scpox")
y = ntuple.array("scpoy")
z = ntuple.array("scpoz")
e = ntuple.array("scene")
mcPDG = ntuple.array("mcpdg")
mcEne = ntuple.array("mcene")

## Binning
binX = np.arange(-40, 41, 5)
binZ = np.arange(-40,41, 5)
binY = np.arange(1800,2050,1)

## 10 slices are defined
#slices = [1810.0, 1860.0, 1870.0, 1880.0, 1892.0, 1905.0, 1918.0,
#          1936.0, 1950.0, 1965.0, 2020.0]

## 5 slices are defined
slices = [1810.0, 1870.0, 1892.0, 1918.0, 1935.0, 2020.0]

nlayers = 5

## Temporary storage for numpy arrays (energy of MCgun, layer information)
e0 = []
l = []

## Start event loop
for i in range(0,Nevents):
    if len(mcPDG[i]) > 7: continue     ## Reject seconday particles and intereactions.
    fig, axs = plt.subplots(nlayers, 1, figsize=(30, 20))

    #keep track of incoming mc particle's energy
    tmp = np.reshape(mcEne[i].take([0]), (1,1))
    e0.append(tmp)


    layers = []
    for j in range(0,nlayers):
        ## find the index falling into each slice category
        idx = np.where((y[i] > slices[j]) & (y[i] <= slices[j+1] ) )

        ## extract x,z and energy (of hits)
        xlayer = x[i].take(idx)[0]
        zlayer = z[i].take(idx)[0]
        elayer = e[i].take(idx)[0]

        ### GeV -- > MeV conversion for cell energies
        elayer = elayer * 1000.00

        ### 2d hist is need for energy weighted distributions
        h0 = axs[j].hist2d(xlayer, zlayer , bins=[binX,binZ], weights=elayer, norm=LogNorm(), cmap=plt.cm.jet)
        layers.append(h0[0])


    ## accumulate for each event
    l.append(layers)
    plt.close(fig)

In [2]:
#Open HDF5 file for writing
hf = h5py.File('gamma-test.hdf5', 'w')
grp = hf.create_group("test")
## convert list --> numpy arrays
layers = np.asarray(l)
e0 = np.reshape(np.asarray(e0),(-1,1))

## write to hdf5 files
grp.create_dataset('energy', data=e0)
grp.create_dataset('layers', data=layers)

hf.close()

In [8]:
f = h5py.File('gamma-test.hdf5', 'r')["test/layers"][:]

In [17]:
f.shape

(980, 5, 16, 16)